# STATS507 HW 2
### **Mengtong Hu**  
### *September 30, 2021*

In [160]:
import numpy as np
import pandas as pd
from timeit import Timer
from collections import defaultdict
from IPython.core.display import display, HTML

### Question 0
## a. 
The code takes a list of tuples and returns the tuple with the largest ending value/or the $n$th values, grouping by the same starting value/or the $m$th value
## b. There are a few errors in the code snippet. First, the indentation of
1. Line 4 and 5 should be aligned. 
2. in line 7, the second index cannot exceed 2 which is one less than the length of a tuple from the sample_list.
3. The inner loop can start with m+1 instead of going through all tuples in the list. The current implementation is inefficient where same comparisons are considered twice. 
4. The naming of variables in the code does not convey much useful information on what they serve for and can be improved by using more meaningful names.

### Question 1

In [126]:
def generate_n_tuplesofk(n, k=4, low=1 , high=100):
    """
    Return a list of n k-tuples of integers between low and high 

    Parameters
    ----------
    n : number of k-tuples 
    
    k : number of elements in a tuple, Default = 4
    
    low : lower bound of an integer value in the tuple, Default =1
    
    high : upper bound of an integer value in the tuple, Default = 100


    Returns
    -------
    result : a list of n k-tuples
          
    """
    rng = np.random.default_rng(seed=1055)
    result = [tuple(rng.integers(low=0, high=10, size=k)) for _ in range(n)]
    return(result)

Test if `generate_n_tuplesofk` returns a list of tuples

In [127]:
tuple_lst = generate_n_tuplesofk(5)

In [131]:
assert type(tuple_lst) is list and all(type(ele) is tuple for 
                                       ele in tuple_lst)

Assert statement returns no errors, meaning the function returns a list of tuples

### Question 2:
 Part a

In [135]:
def get_tuple_with_highest_value_by_idx(sample_list, low_idx, high_idx):
    """
    The code takes a list of tuples and returns 
    the tuple with the largest element at the high_idx,
    grouping by the elements at the low_idx. The method
    is modified based on the orignal snippet

    Parameters
    ----------
    sample_list: a list of tuples with the same length
    
    low_idx : the idx for grouping by
    
    high_idx : the idx at which tuples are compared

    Returns
    -------
    result : a list of selected tuples
          
    """
    op = []
    for m in range(len(sample_list)):
            li = [sample_list[m]]
            for n in range(len(sample_list)):
                if (sample_list[m][low_idx] == sample_list[n][low_idx] and
                        sample_list[m][high_idx] != sample_list[n][high_idx]):
                        li.append(sample_list[n])
            op.append(sorted(li, key=lambda dd: dd[2], reverse=True)[0])
    res = list(set(op))
    return(res)
sample_list = [(1, 3, 7), (0, 1, 2), (1, 9, 8),
               (1,3,4),(1,9,9),(1,2,3),(1,7,9)]
print( "The result is :"
      ,get_tuple_with_highest_value_by_idx(sample_list, 0 ,2))

The result is : [(1, 9, 9), (1, 7, 9), (0, 1, 2)]


part b

In [137]:
def get_tuple_with_highest_value_by_idx_improved(sample_list, low_idx,
                                                 high_idx):
    """
    The code takes a list of tuples and returns 
    the tuple with the largest element at the high_idx,
    grouping by the elements at the low_idx. The method is 
    an improved version based on 
    `get_tuple_with_highest_value_by_idx`
    More efficient for-loop structures are considered

    Parameters
    ----------
    sample_list: a list of tuples with the same length
    
    low_idx : the idx for grouping by
    
    high_idx : the idx at which tuples are compared

    Returns
    -------
    result : a list of selected tuples
    """      
    n = len(sample_list)
    result = []
    visited=set()
    for i in range(n):
        if sample_list[i][low_idx] not in visited:
            visited.add(sample_list[i][low_idx])
            current = [sample_list[i]]
            for j in range(i+1,n):
                if current[-1][low_idx] == sample_list[j][low_idx]:
                    if current[-1][high_idx] < sample_list[j][high_idx]:
                        current = [sample_list[j]]
                    if current[-1][high_idx] == sample_list[j][high_idx]:
                        current.append( sample_list[j]) 
            
            result+=list(set(current))
    
    
    return(result)
print("The result is: "
      , get_tuple_with_highest_value_by_idx_improved(sample_list, 0 ,2))

The result is:  [(1, 9, 9), (1, 7, 9), (0, 1, 2)]


Part c

In [139]:
def get_tuple_with_highest_value_by_idx_o_n(sample_list, low_idx, high_idx):
    """
    The code takes a list of tuples and returns 
    the tuple with the largest element at the high_idx,
    grouping by the elements at the low_idx. The method 
    uses dictionary as data structure which has complexity of
    'O(n)'.

    Parameters
    ----------
    sample_list: a list of tuples with the same length
    
    low_idx : the idx for grouping by
    
    high_idx : the idx at which tuples are compared

    Returns
    -------
    result : a list of selected tuples
    """    
    result = {}
    for i in range(len(sample_list)):
        cur_low_value = sample_list[i][low_idx]
        if cur_low_value not in result:
            result[cur_low_value] = [sample_list[i]]
        else: 
            if sample_list[i][high_idx] > result[cur_low_value][-1][high_idx]:
                result[cur_low_value] = [sample_list[i]] 
            elif sample_list[i][high_idx] == result[cur_low_value
                                                    ][-1][high_idx]: 
                result[cur_low_value].append(sample_list[i])
    
    return([tup for lst in result.values() for tup in lst])
print("The result is: "
      , get_tuple_with_highest_value_by_idx_o_n(sample_list, 0 ,2))


The result is:  [(1, 9, 9), (1, 7, 9), (0, 1, 2)]


In the monte-carlo experiment, I randomly generated 100 tuple lists with 5 tuples in each and tested for the running times of the three methods. The results ouput the average runtime and variance for 100 monte-carlo expeirments.



In [142]:
tuple_lst = generate_n_tuplesofk(10)
low_idx = 0
high_idx = 2
n = 10
function_lst = ['Original Snippet O(n^2)', 'Improved Snippet O(n^2)',
                'Improved Method O(n)' ]
time_lst_mean = defaultdict(list)
time_lst_n = defaultdict(list)
for rep in range(100):
    rnd_lst = generate_n_tuplesofk(n)
    for idx , f in enumerate([get_tuple_with_highest_value_by_idx, 
         get_tuple_with_highest_value_by_idx_improved, 
         get_tuple_with_highest_value_by_idx_o_n]):
        t = Timer ('f(sample, a, b)', globals={'f':f, 'sample':rnd_lst,
                                           'a':low_idx, 'b': high_idx})
        
        time_lst_n[function_lst[idx]].append(t.timeit())
for func_name in function_lst:
    time_lst_mean[func_name] = [np.mean(time_lst_n[func_name])]
    time_lst_mean[func_name].append(np.var(time_lst_n[func_name]))

In [143]:
time_lst_mean_tb = pd.DataFrame.from_dict(time_lst_mean, orient = 'index',
                                         columns=['mean runtime', 'variance'])
display(HTML(time_lst_mean_tb.to_html()))

,mean runtime,variance
Original Snippet O(n^2),25.269254,0.212552
Improved Snippet O(n^2),10.746367,0.061537
Improved Method O(n),4.167102,0.012206


Out of 100 monte-carlo expeirments of tuple size equal to 10, the run time of the Improved Method has less runtime compared to the orignal snippet and the improved snippet. The variance is within acceptable range.

### Question 3:

In [144]:
fields = ['SEQN', 'RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR', 
          'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR' ]


In [145]:
# Read the dataset with the specified fields
df1112 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT')[fields]
df1112['cohort'] = '2011-2012'
df1314 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT')[fields]
df1314['cohort'] = '2013-2014'
df1516 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT')[fields]
df1516['cohort'] = '2015-2016'

df1718 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT')[fields]
df1718['cohort'] = '2017-2018'
Demo = df1112.append(df1314).append(df1516).append(df1718)

In [146]:
# Rename variables
Demo = Demo.rename({'SEQN' : 'id' , 'RIDAGEYR' : 'age','RIDRETH3' : 'race and ethnicity'
             ,'DMDEDUC2' : 'education','DMDMARTL' : 'marital status',
             'RIDSTATR' : 'interview exam status', 'SDMVPSU' : 'psu', 
             'SDMVSTRA' : 'stratum', 'WTMEC2YR' : 'two year mec wt' , 'WTINT2YR' :
            'twoyr_interview_wt'}, axis = 1)
# Changed variables for appropriate category names
Demo['race and ethnicity'] = pd.Categorical(Demo['race and ethnicity'].replace(
    {1: 'Mexican American', 
     2: 'Other Hispanic', 
     3: 'Non-Hispanic White',
     4: 'Non-Hispanic Black',
     6: 'Non-Hispanic Asian',
     7: 'Other Race - Including Multi-Racial',
    -1: 'Missing'}))

Demo['education'] = pd.Categorical(Demo['education'].replace(
    {1: 'Less than 9th grade', 
     2: '9-11th grade (Includes 12th grade with no diploma)', 
     3: 'High school graduate/GED or equivalent',
     4: 'Some college or AA degree',
     5: 'College graduate or above',
     7: 'Refused',
     9: 'Unknown',
    -1: 'Missing'}))

Demo['marital status'] = pd.Categorical(Demo['marital status'].replace(
    {1: 'Married', 
     2: 'Widowed', 
     3: 'Divorced',
     4: 'Separated',
     5: 'Never married',
     6: 'Living with partner',
     77: 'Refused',
     99: 'Unknown',
    -1: 'Missing'}))

Demo['interview exam status'] = pd.Categorical(Demo['interview exam status'].replace(
    {1: 'Interviewed only', 
     2: 'Both interviewed and MEC examined', 
    -1: 'Missing'}))

In [147]:
# Change the rest of the float varaibels into integer variables 
fields_to_int = ['id', 'age', 'psu', 'stratum']
Demo[fields_to_int] =Demo[fields_to_int].applymap(np.int32)
Demo.to_pickle('./Demo_pickle.pkl')

The Denitition Dataset:

In [148]:
# Read the dataset 
df1112_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/OHXDEN_G.XPT')

/usr/local/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [149]:
# Specify the fields we are interested in
fields_dent = ['SEQN','OHDDESTS'] + [ele for ele in df1112_dent.columns.values[4:64]]

In [150]:
df1112_dent = df1112_dent[fields_dent]
df1314_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT')[fields_dent]

In [151]:
df1516_dent = pd.read_sas(
      'https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/OHXDEN_I.XPT')[fields_dent]

In [152]:
df1718_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/OHXDEN_J.XPT')[fields_dent]

In [153]:
# Add an indictor 'cohort' for the year of the sampling
df1112_dent['cohort'] = '2011-2012'
df1314_dent['cohort'] = '2013-2014'
df1516_dent['cohort'] = '2015-2016'
df1718_dent['cohort'] = '2017-2018'

In [154]:
Dent = df1112_dent.append(df1314_dent).append(df1516_dent).append(df1718_dent)

In [155]:
# Change the data column names to lower case 
Dent.columns = Dent.columns.str.lower()

In [156]:
# Create naming dictionaries for all data column 
d1 = {'seqn':'id','ohddests':'dentition status code'}
d2 = dict(zip(Dent.columns[2:34], 
             ['tooth count {}'.format(x+1) for x in range(len(Dent.columns[2:34]))]))

d3 = dict(zip(Dent.columns[34:48], 
             ['Coronal Caries Tooth Count {}'.format(x+2) for x in range(len(Dent.columns[34:48]))]))

d4 = dict(zip(Dent.columns[49:64], 
             ['Coronal Caries Tooth Count {}'.format(x) for x in range(19,32)]))
d1.update(d2)
d1.update(d3)
d1.update(d4)
# Rename the data columns
Dent = Dent.rename(d1,axis=1)

In [157]:
# Add category names for the categorical variables
Dent['dentition status code'] = pd.Categorical(Dent['dentition status code']
                            .replace({1: 'Complete', 2: 'Partial', 3:'Not Done'}))
for i in range(2, 34):
    Dent[str(Dent.columns[i])] = pd.Categorical(Dent[str(Dent.columns[i])].replace(
        {1: 'Primary tooth (deciduous) present',
         2: 'Permanent tooth present', 
         3: 'Dental implant',
         4: 'Tooth not present', 
         5: 'Permanent dental root fragment present',
         9: 'Could not assess',
        -1: 'Missing'}))

for i in range(34, 62):
    Dent[str(Dent.columns[i])] = pd.Categorical(Dent[str(Dent.columns[i])].replace(
        {b'D': 'Sound primary tooth',
         b'E': 'Missing due to dental disease', 
         b'J': 'Permanent root tip is present but no restorative replacement is present',
         b'K': 'Primary tooth with surface condition (s)', 
         b'M': 'Missing due to other causes',
         b'P': 'Missing due to dental disease but replaced by a removable restoration',
         b'Q': 'Missing due to other causes but replaced by a removable restoration',
         b'R': 'Missing due to dental disease, but replaced by a fixed restoration',
         b'S': 'Sound permanent tooth',
         b'T': 'Permanent root tip is present but a restorative replacement is present',
         b'U': 'Unerupted',
         b'X': 'Missing due to other causes, but replaced by a fixed restoration',
         b'Y': 'Tooth present, condition cannot be assessed',
         b'Z': 'Permanent tooth with surface condition (s)',
         b'': 'Missing'}))

In [158]:
# Write to a pickle object
fields_to_int = ['id']
Dent[fields_to_int] =Dent[fields_to_int].applymap(np.int32)
Dent.to_pickle('./Dent_pickle.pkl')

In [159]:
print('The number of cases in the NHANES demographic ',
      'datasets from 2011 to 2018 is:',Demo['id'].nunique())
print('The number of cases in the NHANES oral health ', 
      'and dentition datasets from 2011 to 2018 is:',Dent['id'].nunique())

The number of cases in the NHANES demographic  datasets from 2011 to 2018 is: 39156
The number of cases in the NHANES oral health  and dentition datasets from 2011 to 2018 is: 35909
